In [1]:
import pandas as pd
import numpy as np


In [55]:
emails = pd.read_csv("emails-small.csv")

TypeError: read_csv() got an unexpected keyword argument 'index'

In [3]:
#emails = pd.read_csv("emails.csv")
#df = emails.iloc[0:1000,:]
#df.to_csv("emails-small.csv")

In [54]:
emails

,Unnamed: 0,file,message
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...,...
995,995,allen-p/all_documents/458.,Message-ID: <20430828.1075855696096.JavaMail.e...
996,996,allen-p/all_documents/459.,Message-ID: <18425275.1075855696118.JavaMail.e...
997,997,allen-p/all_documents/46.,Message-ID: <24036204.1075855666506.JavaMail.e...
998,998,allen-p/all_documents/460.,Message-ID: <33307764.1075855696139.JavaMail.e...


In [4]:
emails.shape

(1000, 3)

In [29]:
email = emails["message"][600].split("\n")

In [43]:
emails["message"]

0      Message-ID: <18782981.1075855378110.JavaMail.e...
1      Message-ID: <15464986.1075855378456.JavaMail.e...
2      Message-ID: <24216240.1075855687451.JavaMail.e...
3      Message-ID: <13505866.1075863688222.JavaMail.e...
4      Message-ID: <30922949.1075863688243.JavaMail.e...
                             ...                        
995    Message-ID: <20430828.1075855696096.JavaMail.e...
996    Message-ID: <18425275.1075855696118.JavaMail.e...
997    Message-ID: <24036204.1075855666506.JavaMail.e...
998    Message-ID: <33307764.1075855696139.JavaMail.e...
999    Message-ID: <15009418.1075855696162.JavaMail.e...
Name: message, Length: 1000, dtype: object

In [30]:
email

['Message-ID: <23615201.1075855687408.JavaMail.evans@thyme>',
 'Date: Fri, 20 Oct 2000 03:06:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: stagecoachmama@hotmail.com',
 'Subject: ',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: stagecoachmama@hotmail.com',
 'X-cc: ',
 'X-bcc: ',
 "X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 'Lucy,',
 '',
 '       Here are the actual utility bills versus the cap.  Did we collect ',
 "these overages?  Let's discuss further?  Remember these bills were paid in ",
 'July and August.  The usage dates are much earlier.  I have the bills but I ',
 'can get them to you if need be.',
 '',
 'Philip']

In [45]:
def create_variables(data_message):
    email = data_message.split("\n")
    end = False
    message = []

    for iteration,part in enumerate(email):
        if end == False:
            if part.find(":"):
                if part.startswith("Date:"):
                    Date = part.split(": ")[1]
                if part.startswith("From:"):
                    From = part.split(": ")[1]
                if part.startswith("To:"):
                    To = part.split(": ")[1]
                if part.startswith("Subject:"):
                    Subject = part.split(": ")[1]
                if part.startswith("X-FileName:"):
                    end = True
        else:
            message.append(part)

    Message = "\n".join(message)

    return From,To,Date, Subject, Message
                


In [48]:
From,To,Date, Subject, Message = create_variables(emails["message"][500])

In [49]:
print(Date)
print(From)
print(To)
print(Subject)
print(Message)

Thu, 25 Jan 2001 00:17:00 -0800 (PST)
phillip.allen@enron.com
mike.grigsby@enron.com
Re

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 01/25/2001 
08:17 AM ---------------------------
From: Leslie Lawner@ENRON on 01/24/2001 08:17 PM CST
To: MBD <MDay@GMSSR.com>
cc: Harry Kingerski/NA/Enron@Enron, Jeff Dasovich/NA/Enron@Enron, James D 
Steffes/NA/Enron@Enron, Phillip K Allen/HOU/ECT@ECT, Don Black/HOU/EES@EES, 
James Shirley/HOU/EES@EES, Frank Ermis/HOU/ECT@ECT, Paul Kaufman/PDX/ECT@ECT 
Subject: Re: Draft of Opposition to ORA/TURN petition  

Everything is short and sweet except the caption!  One comment.  The very 
last sentence reads :  PG&E can continue to physically divert gas if 
necessary . . . "  SInce they haven't actually begun to divert yet, let's 
change that sentence to read "PG&E has the continuing right to physically 
divert gas if necessary..."  

I will send this around for comment.  Thanks for your promptness.

Any comments, anyone?



	MBD <MDay@GMSSR

In [51]:
emails

,Unnamed: 0,file,message
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...,...
995,995,allen-p/all_documents/458.,Message-ID: <20430828.1075855696096.JavaMail.e...
996,996,allen-p/all_documents/459.,Message-ID: <18425275.1075855696118.JavaMail.e...
997,997,allen-p/all_documents/46.,Message-ID: <24036204.1075855666506.JavaMail.e...
998,998,allen-p/all_documents/460.,Message-ID: <33307764.1075855696139.JavaMail.e...
